In [1]:
import json
import pandas as pd
import glob
import numpy as np

In [2]:
data_in = "../../sih-raw-data/geojson/"
data_out = "../data/geojson/"

## fix geojson

In [3]:
with open(data_in + r"state.json", "r") as f:
    states = json.load(f)
for i in states["features"]:
    v = i["properties"]["NAME_1"].lower()
    if v == "nct of delhi":
        i["id"] = "delhi"
    elif v == "jammu and kashmir":
        i["id"] = "jammu & kashmir"
    elif v == "puducherry":
        i["id"] = "pondicherry"
    elif v == "andaman and nicobar":
        i["id"] = "andaman & nicobar"
    elif v == "dadra and nagar haveli":
        i["id"] = "dadra and nagar hav"
    elif v == "tamil nadu":
        i["id"] = "tamilnadu"
    else:
        i["id"] = v
with open(data_out + r"state_id.json", "w") as f:
    json.dump(states, f)

In [4]:
with open(data_in + r"district.json", "r") as f:
    districts = json.load(f)
for i in districts["features"]:
    v = i["properties"]["NAME_2"].lower()
    if v == "allahabad":
        i["id"] = "prayagraj"
    elif v == "mumbai city":
        i["id"] = "mumbai"
    elif v == "north and middle andaman":
        i["id"] = "north & middle andaman"
    elif v == "dadra and nagar haveli":
        i["id"] = "dadra & nagar haveli"
    elif v == "lakhimpur kheri":
        i["id"] = "kheri"
    elif v == "barabanki":
        i["id"] = "bara banki"
    elif v == "dantewada":
        i["id"] = "dakshin bastar dantewada"
    elif v == "dahod":
        i["id"] = "dohad"
    elif v == "chikballapura":
        i["id"] = "chikkaballapura"
    elif v == "amroha":
        i["id"] = "jyotiba phule nagar"
    elif v == "kaimur":
        i["id"] = "kaimur (bhabua)"
    elif v == "kasganj":
        i["id"] = "kansiram nagar"
    elif v == "hathras":
        i["id"] = "mahamaya nagar"
    elif v == "ranga reddy":
        i["id"] = "rangareddy"
    elif v == "sant ravi das nagar":
        i["id"] = "sant ravi das nagar(bhadohi)"
    elif v == "saran":
        i["id"] = "saran (chhapra)"
    elif v == "shravasti":
        i["id"] = "shrawasti"
    elif v == "nellore":
        i["id"] = "sri potti sriramulu nellore"
    elif v == "faizabad":
        i["id"] = "ayodhya"
    elif v == "morigaon":
        i["id"] = "marigaon"

    else:
        i["id"] = v
with open(data_out + r"district_id.json", "w") as f:
    json.dump(districts, f)

In [5]:
with open(data_in + r"india.json", "r") as f:
    india = json.load(f)
for i in india["features"]:
    i["id"] = "india"
with open(data_out + r"india_id.json", "w") as f:
    json.dump(india, f)

In [6]:
with open(data_in + r"block.json", "r") as f:
    blocks = json.load(f)
for i in blocks["features"]:
    v = i["properties"]["NAME_3"].lower()
    if v == "allahabad":
        i["id"] = "prayagraj"
    else:
        i["id"] = v
with open(data_out + r"block_id.json", "w") as f:
    json.dump(blocks, f)

In [7]:
with open("../data/geojson/district_id.json", "r") as f:
    districts_json = json.load(f)
for f in glob.glob(data_in + "delhi-districts/*"):
    district_name = f.split("/")[-1].split(".")[0]
    with open(f) as file:
        geojson = json.load(file)
    geojson["features"][0]["id"] = district_name
    if district_name != "west":
        districts_json["features"].append(geojson["features"][0])
    else:
        for i in range(len(districts_json["features"])):
            if districts_json["features"][i]["id"] == "west":
                districts_json["features"][i] = geojson["features"][0]
        #         print(geojson['features'][0])

In [8]:
with open("../data/geojson/district_id.json", "w") as f:
    json.dump(districts_json, f)

## Find not found and save

In [3]:
with open(data_out + r"district_id.json", "r") as f:
    districts_json = json.load(f)
with open(data_out + r"block_id.json", "r") as f:
    blocks_json = json.load(f)

In [4]:
df = pd.read_parquet(r"../data/comp/categories.parquet.gzip")

In [5]:
df = pd.read_parquet(r"../data/comp/gw-block-pre-post.parquet.gzip")

In [5]:
districts = list(set(list(df["district"])))
blocks = list(set(list(df["block"])))

In [6]:
districts_not_found = []
blocks_not_found = []

In [7]:
for i in districts:
    found = False
    for j in districts_json["features"]:
        if j["id"] == i:
            found = True
            break
    if not found:
        districts_not_found.append(i)
df_not_districts = pd.DataFrame({"districts": districts_not_found})

In [8]:
df_not_districts.sort_values("districts", inplace=True)

In [9]:
for i in blocks:
    found = False
    for j in blocks_json["features"]:
        if j["id"] == i:
            found = True
            break
    if not found:
        blocks_not_found.append(i)
df_not_blocks = pd.DataFrame({"blocks": blocks_not_found})
df_not_blocks.sort_values("blocks", inplace=True)

In [11]:
ite_v = 0
for i in blocks_not_found:
    t = (
        df[df["block"] == i]
        .groupby([ "state", "district", "block"])
        .count()
        .reset_index().loc[:,['state','district','block']]
    )
    if ite_v >= 1:
        t_ans=t_ans.append(t, ignore_index=True)
    else:
        t_ans = t
        ite_v += 1
        
#         print(t,t_ans)
#         break

In [12]:
len(blocks_not_found)

5214

In [13]:
t_ans.to_csv(r"../data/blocks_not_found_categories.csv", index=False)

In [14]:
t_ans.shape

(5368, 3)

In [15]:
df_not_districts.to_csv(r"../data/districts_not_found_categories.csv", index=False)

In [16]:
block = []
for j in blocks_json["features"]:
    block.append(j["id"])
all_blocks_json = pd.DataFrame({"blocks": block})
all_blocks_json.sort_values("blocks", inplace=True)

In [17]:
all_blocks_json.to_csv(r"../data/all-blocks-json.csv", index=False)

In [18]:
district = []
for j in districts_json["features"]:
    district.append(j["id"])
all_districts_json = pd.DataFrame({"districts": district})
all_districts_json.sort_values("districts", inplace=True)

In [19]:
all_districts_json.to_csv(r"../data/all-districts-json.csv", index=False)

## Rough

In [22]:
with open(data_out + r"district_id.json", "r") as f:
    districts_json = json.load(f)
with open(data_out + r"block_id.json", "r") as f:
    blocks_json = json.load(f)

In [23]:
df = pd.read_csv(r"../data/districts_not_found.csv")

In [24]:
df

,districts
0,j&k district 1


In [28]:
for i in districts_json["features"]:
    if i["id"].lower() == "west":
        print(i)

{'type': 'Feature', 'properties': {}, 'geometry': {'type': 'Polygon', 'coordinates': [[[76.95751190185547, 28.684564054643918], [76.95744752883911, 28.684413460213964], [76.95736169815062, 28.684262865567455], [76.95699691772461, 28.683660484815782], [76.95562362670898, 28.670708460642608], [76.95493698120117, 28.670407231724507], [76.95545196533203, 28.66920230739231], [76.97982788085938, 28.654591497496344], [76.98532104492188, 28.654591497496344], [76.9951057434082, 28.660315562573203], [77.00077056884764, 28.655043408733377], [77.00557708740234, 28.651277422243783], [77.00540542602539, 28.636212123876533], [77.00763702392578, 28.634705475045894], [77.01107025146484, 28.62762393578897], [77.01759338378906, 28.62415832756724], [77.02360153198242, 28.626267841849668], [77.02857971191406, 28.629281360145207], [77.0313262939453, 28.62943203378891], [77.03235626220703, 28.632294791921947], [77.03493118286133, 28.630034726200766], [77.03681945800781, 28.627925287618567], [77.0400810241699